In [66]:
%pip install --user pymilvus "pymilvus[model]" sentence-transformers nltk FlagEmbedding ipywidgets
%pip install --user --upgrade jupyter ipywidgets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
import os
import pandas as pd

dfs = []
for file in os.listdir("dataset"):
    hr_name = file.replace(".pdf.xlsx", "")
    file_df = pd.read_excel(f"dataset/{file}", header=0, names=["text", "page"])
    file_df["page"] = file_df["page"] + 1
    file_df["file"] = hr_name
    file_df[["text"]] = file_df[["text"]].apply(lambda row: row.str.replace(r'[^\w.()«»\-– ]+', " ", regex=True))
    file_df[["text"]] = file_df[["text"]].apply(lambda row: row.str.replace(r'\s+', " ", regex=True))
    dfs.append(file_df)
df = pd.concat(dfs)
df = df.dropna()

In [31]:
from pymilvus import model
from tqdm import tqdm

# embedding_fn = model.hybrid.BGEM3EmbeddingFunction(use_fp16=False, device="cpu")
embedding_fn = model.dense.SentenceTransformerEmbeddingFunction(
    model_name='BAAI/bge-m3', # Specify the model name
    device='cpu' # Specify the device to use, e.g., 'cpu' or 'cuda:0'
)

docs = df["text"].str[:8192].values.tolist()
vectors = []# embedding_fn.encode_documents(tqdm(docs))
for doc in tqdm(docs):
    vectors.append(embedding_fn.encode_documents([doc])[0])
print("Dim:", embedding_fn.dim, vectors[0].shape)  # Dim: 768 (768,)

# Each entity has id, vector representation, raw text, and a subject label that we use
# to demo metadata filtering later.
data = [
    {"id": i, "vector": vectors[i], "text": df.iloc[i]["text"], "page": df.iloc[i]["page"], "file": df.iloc[i]["file"]}
    for i in range(len(vectors))
]

print("Data has", len(data), "entities, each with fields: ", data[0].keys())
print("Vector dim:", len(data[0]["vector"]))

c:\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
100%|██████████| 1050/1050 [04:40<00:00,  3.74it/s]


Dim: 1024 (1024,)
Data has 2415 entities, each with fields:  dict_keys(['id', 'vector', 'text', 'page', 'file'])
Vector dim: 1024


In [59]:
df_data = pd.DataFrame(data)

In [77]:
data

[{'id': 0,
  'vector': array([-0.02835579,  0.01825912, -0.05112049, ..., -0.01695435,
          0.02887001, -0.02505875], dtype=float32),
  'text': 'Проект D-1C1-1 «Развитие унифицированной отраслевой платформы «1С ERP 2.0 ЦИФРОВОЙ РОСАТОМ» Инструкция пользователей «D-1C1-1.05.1 Планирование работы ТС»',
  'page': 1,
  'file': 'D-1C1-1.05.01 Планирование работы ТС'},
 {'id': 1,
  'vector': array([ 0.00954152, -0.02555463, -0.04228625, ...,  0.00806488,
         -0.03042443, -0.03089831], dtype=float32),
  'text': ' ЛИСТ ИЗМЕНЕНИЙ Дата Версия Краткое описание изменений ФИО 30.10.2020 1 Первая версия документа Маркин С.С. 04.10.2023 2 Перенос в новый шаблон актуализация Митрофанов В.Р. ',
  'page': 2,
  'file': 'D-1C1-1.05.01 Планирование работы ТС'},
 {'id': 2,
  'vector': array([-0.03441276, -0.01681485, -0.04898072, ...,  0.00298577,
          0.02404443, -0.0560878 ], dtype=float32),
  'text': ' открывшемся справочнике необходимо нажать на кнопку создания нового эле- мента. Сценарий

In [18]:
from pymilvus import MilvusClient

client = MilvusClient("http://docker.home:19531")

In [19]:
# client.load_collection("guides_collection")
client.list_collections()

['demo_collection', 'guides', 'guides_collection']

In [85]:
from pymilvus import DataType

schema = MilvusClient.create_schema(
    auto_id=False,
    enable_dynamic_field=True,
)

# 3.2. Add fields to schema
schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="page", datatype=DataType.INT64)
schema.add_field(field_name="text", datatype=DataType.VARCHAR, max_length=8192)
schema.add_field(field_name="file", datatype=DataType.VARCHAR, max_length=256)
schema.add_field(field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=1024)
index_params = MilvusClient.prepare_index_params()

index_params.add_index(
    field_name="vector",
    metric_type="COSINE",
    index_type="IVF_FLAT",
    index_name="vector_index",
    params={ "nlist": 128 }
)


In [87]:
client.drop_collection("guides_collection")
client.create_collection(
    collection_name="guides_collection",
    schema=schema,
    index_params=index_params,
    dimension=1024,  # The vectors we will use in this demo has 768 dimensions
)

In [88]:
for i in range(0, len(data), 100):
    res = client.insert(collection_name="guides_collection", data=data[i:i+100])

print(res)

{'insert_count': 15, 'ids': [2400, 2401, 2402, 2403, 2404, 2405, 2406, 2407, 2408, 2409, 2410, 2411, 2412, 2413, 2414]}


In [3]:
{"a": "b"} | {"b": "b"}

{'a': 'b', 'b': 'b'}

In [93]:
query_vectors = embedding_fn.encode_queries(["Как создать справочник номенклатуры?"])
# If you don't have the embedding function you can use a fake vector to finish the demo:
# query_vectors = [ [ random.uniform(-1, 1) for _ in range(768) ] ]
client.load_collection("guides_collection")
res = client.search(
    collection_name="guides_collection",  # target collection
    data=query_vectors,  # query vectors
    limit=15,  # number of returned entities
    output_fields=["text", "file", "page"],  # specifies fields to be returned
)
res[0]

[{'id': 1270,
  'distance': 0.6212438344955444,
  'entity': {'file': 'Инструкция D-1C1-1.10.19 Формирование налоговой декларации по НП-v3',
   'page': 41,
   'text': ' 1.3.3.3 Справочник «Номенклатура» Справочник предназначен для хранения информации о номенклатурных позициях следующих типов товар услуга работа тара и набор. . Для автоматизации заполнения Приложения 1 графы 3 4 (код 770) в карточке номенклатуры с Видом «Спец.оснастка и спец.одежда на складе» на вкладке «Реквизиты» в раздел «Дополнительные реквизиты» добавлен новый реквизит «Форменная одежда» (тип «Булево»). Реквизит является видимым только для данного Вида номенклатуры(Рисунок 7). При записи новой карточки с видом «Спец.оснастка и спец.одежда на складе» система выводит сообщение «Данная одежда является форменной » и если ответ да то реквизит будет заполнен значением «Истина». 1.4 Формирование декларации по налогу на прибыль. Ответственный исполнитель Бухгалтер по учету налоговых обязательств и формированию отчетности (Н

: 